# Get Routes Table

In this jupyter notebook you will see how I clean data from the "ascents table", this table has a record of the climber's ascension of a route, it also contains more data like location, grade, hard/soft, rating.. Each climber fills it in their particular way, you will see there is a lot to clean. this job is divided in:
1. String cleaning for each column (using class `text_processor_class.py`)
2. Sentiment (first checks language and then if comments are positive or negative, using class `text_processor_class.py`)
3. Splitting
4. Cleaning and filtering outlayers
5. Adding climbers table features (this table was obtained from ascents table as well, how to in [this project](https://github.com/jordi-zaragoza/Climbing-Data-Analysis))
6. Clean and group naming (it compares names and replaces if they are alike, using class `text_processor_class.py`)
    - crag
    - sector
    - route name
    
If you want to know how my text_processr_class works, take a look at `GUIDE.text_processor_class.ipynb`

In [2]:
import pandas as pd 
import numpy as np
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from lib.text_processor_class import *
from lib import clean_jor, tables_jor

In [3]:
pd.set_option("display.max_columns",None)
pd.set_option('display.max_rows', 10)

In [4]:
txt = Text_processor()

Initialized text_processor class


/home/jzar/.local/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.23.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/jzar/.local/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator MultinomialNB from version 0.23.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


## 1. String cleaning

### Get conversions table

In [4]:
conversions = tables_jor.conversion_table()

### Get and clean ascent table

I have to create a routes table where I will select the most valuable features for each route based on user evaluation.

In [67]:
# %%time
# ascent = pd.read_csv('../../databases/ascent.csv',low_memory=False)
# ascent_df = clean_jor.Correct_grade_id(ascent)
# ascent_df.to_csv('../../databases/ascent_grade_correction.csv')
'''
CAUTION:
this piece of code above takes:
CPU times: user 38.3 s, sys: 2.55 s, total: 40.9 s
Wall time: 41.2 s
'''

ascent_df = pd.read_csv('../../databases/ascent_grade_correction.csv',low_memory=False, index_col=0)

### Choosing the correct features

In [68]:
print(ascent_df.shape)
ascent_df.head(3)

(4111878, 28)


,id,user_id,grade_id,notes,raw_notes,method_id,climb_type,total_score,date,year,last_year,rec_date,project_ascent_date,name,crag_id,crag,sector_id,sector,country,comment,rating,description,yellow_id,climb_try,repeat,exclude_from_ranking,user_recommended,chipped
0,2,1.0,36,NaN,0,3,0,545,918342000.0,1999.0,0.0,1.107126e+09,0.0,The King And I,16596.0,Railay,61.0,Dum's kitchen,THA,Thailand\n,0.0,NaN,255.0,0.0,0.0,0.0,0.0,0.0
1,3,1.0,36,NaN,0,3,0,545,925509600.0,1999.0,0.0,1.107126e+09,0.0,vet ej,0.0,Nya berg - segl.,0.0,NaN,NaN,\n,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0
2,4,1.0,36,NaN,0,3,0,545,933026400.0,1999.0,0.0,1.107126e+09,0.0,Mr Big,209.0,Sjöända,371.0,Huvudväggen,SWE,\n,0.0,NaN,255.0,0.0,0.0,0.0,0.0,0.0


In [69]:
ascent_df2 = ascent_df.copy()
ascent_df2 = ascent_df2[ascent_df.exclude_from_ranking == 0]
ascent_df2 = ascent_df2[ascent_df.climb_type == 0] # sportsclimb
ascent_df2 = ascent_df2.drop(columns = ['raw_notes','exclude_from_ranking','climb_type','total_score','date','year','rec_date','project_ascent_date','description','method_id','last_year'])
print("2..", ascent_df2.shape)

<ipython-input-69-f6d93b26d55e>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ascent_df2 = ascent_df2[ascent_df.climb_type == 0] # sportsclimb


2.. (2851212, 17)


In [70]:
ascent_df2.head(3)

,id,user_id,grade_id,notes,name,crag_id,crag,sector_id,sector,country,comment,rating,yellow_id,climb_try,repeat,user_recommended,chipped
0,2,1.0,36,NaN,The King And I,16596.0,Railay,61.0,Dum's kitchen,THA,Thailand\n,0.0,255.0,0.0,0.0,0.0,0.0
1,3,1.0,36,NaN,vet ej,0.0,Nya berg - segl.,0.0,NaN,NaN,\n,0.0,0.0,0.0,0.0,0.0,0.0
2,4,1.0,36,NaN,Mr Big,209.0,Sjöända,371.0,Huvudväggen,SWE,\n,0.0,255.0,0.0,0.0,0.0,0.0


## 2. Clean string columns

We initialize the class text_processor and run the clinning for each column

In [80]:
# %%time
# ascent_df2_2 = ascent_df2.copy()
# ascent_df2_2['name'] = txt.cleaning_function(ascent_df2_2.name)
# ascent_df2_2['sector'] = txt.cleaning_function(ascent_df2_2.sector)
# ascent_df2_2['crag'] = txt.cleaning_function(ascent_df2_2.crag)
# ascent_df2_2['country'] = txt.cleaning_function(ascent_df2_2.country)
'''
CAUTION:
this piece of code above takes:
CPU times: user 4min 34s, sys: 907 ms, total: 4min 34s
Wall time: 4min 37s
'''
ascent_df2_2.head(3)

CPU times: user 4min 34s, sys: 907 ms, total: 4min 34s
Wall time: 4min 37s


,id,user_id,grade_id,notes,name,crag_id,crag,sector_id,sector,country,comment,rating,yellow_id,climb_try,repeat,user_recommended,chipped
0,2,1.0,36,NaN,king and i,16596.0,railay,61.0,dums kitchen,tha,Thailand\n,0.0,255.0,0.0,0.0,0.0,0.0
1,3,1.0,36,NaN,vet ej,0.0,nya berg segl,0.0,nan,nan,\n,0.0,0.0,0.0,0.0,0.0,0.0
2,4,1.0,36,NaN,mr big,209.0,sjoanda,371.0,huvudvaggen,swe,\n,0.0,255.0,0.0,0.0,0.0,0.0


## 3. Check if the coments are positive or negetive

We want this to run it once as it will take a long time

In [81]:
# %%time
# from polyglot.detect.base import logger
# # we disable the warning messages
# logger.disabled = True
# ascent_df2_2['sentiment'] = ascent_df2_2.comment.apply(lambda x: txt.comment_sentiment(x) if len(str(x))>2  else 0)
# ascent_df2_2['comment_bool'] = ascent_df2_2.comment.apply(lambda x: 0 if len(str(x))<4  else 1)
# logger.disabled = False
# ascent_df2_2.to_csv('../../databases/ascent_with_sentiment.csv')
'''
CAUTION:
this piece of code above takes:
CPU times: user 2min 3s, sys: 1.55 s, total: 2min 4s
Wall time: 2min 6s
'''

ascent_df2_2 = pd.read_csv('../../databases/ascent_with_sentiment.csv',low_memory=False, index_col=0)
ascent_df2_2.head(3)

CPU times: user 2min 3s, sys: 1.55 s, total: 2min 4s
Wall time: 2min 6s


,id,user_id,grade_id,notes,name,crag_id,crag,sector_id,sector,country,comment,rating,yellow_id,climb_try,repeat,user_recommended,chipped,sentiment,comment_bool
0,2,1.0,36,NaN,king and i,16596.0,railay,61.0,dums kitchen,tha,Thailand\n,0.0,255.0,0.0,0.0,0.0,0.0,0.0,1
1,3,1.0,36,NaN,vet ej,0.0,nya berg segl,0.0,NaN,NaN,\n,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,4,1.0,36,NaN,mr big,209.0,sjoanda,371.0,huvudvaggen,swe,\n,0.0,255.0,0.0,0.0,0.0,0.0,0.0,0


##  4. Continue cleaning

Here we split notes column into easy, hard, first_ascent, traditional.. and rearrange the grading based on that (+1 if hard, -1 if soft.. that is a "/" grade, like from 7a to 7a/)

In [83]:
ascent_df2_2.notes.value_counts(dropna = False).head(5)

NaN            2128264
Second Go       265786
Soft            155117
Hard            128852
Traditional      62667
Name: notes, dtype: int64

### Splitting

In [84]:
# %%time
# #Add a bool for rating
# ascent_df2_2['rating_bool'] = ascent_df2_2.rating.apply(lambda x: 0 if x == 0  else 1)
# # Split the notes column
# ascent_df3 = ascent_df2_2.copy()
# ascent_df3.notes = ascent_df3.notes.fillna('-')
# ascent_df3.reset_index(drop = True, inplace = True)
# notes = ascent_df3.notes.apply(clean_jor.Split_notes)
# notes_df = pd.DataFrame(notes.to_list(), columns = ['first_ascent', 'soft', 'hard', 'traditional'])
# ascent_df3 = pd.concat([ascent_df3,notes_df], axis = 1)
# ascent_df3 = ascent_df3.drop(columns = ['notes'])
# ascent_df3.grade_id = ascent_df3.apply(clean_jor.Easy_hard, axis = 1)   
# print(ascent_df3.shape)
# ascent_df3.to_csv('../../databases/ascent_df3.csv')
'''
The piece of code above takes:
CPU times: user 1min 26s, sys: 2.42 s, total: 1min 29s
Wall time: 1min 30s
'''

ascent_df3 = pd.read_csv('../../databases/ascent_df3.csv',low_memory=False, index_col=0)

(2851212, 23)
CPU times: user 1min 30s, sys: 4.09 s, total: 1min 34s
Wall time: 1min 36s


In [85]:
ascent_df3.head(3)

,id,user_id,grade_id,name,crag_id,crag,sector_id,sector,country,comment,rating,yellow_id,climb_try,repeat,user_recommended,chipped,sentiment,comment_bool,rating_bool,first_ascent,soft,hard,traditional
0,2,1.0,36,king and i,16596.0,railay,61.0,dums kitchen,tha,Thailand\n,0.0,255.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,0,0
1,3,1.0,36,vet ej,0.0,nya berg segl,0.0,NaN,NaN,\n,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0
2,4,1.0,36,mr big,209.0,sjoanda,371.0,huvudvaggen,swe,\n,0.0,255.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0


### Filtering task

In [86]:
# Grade below 9c
ascent_df4 = ascent_df3.copy()
ascent_df4 = ascent_df4[ascent_df4['grade_id'] < 79] # below 9c
ascent_df4 = ascent_df4[ascent_df4['grade_id'] > 28] # # above 5a
ascent_df4.dropna(subset=['name'],inplace = True)
ascent_df4.reset_index(drop = True, inplace = True)
print("4..",ascent_df4.shape)

4.. (2737449, 23)


#### - Name column

In [87]:
ascent_df4.name.value_counts().head()

dont know name    3751
no name           3096
route             3003
unknown           2768
n                 2023
Name: name, dtype: int64

In [88]:
# filter the weird naming
ascent_df5 = ascent_df4.copy()
ascent_df5 = ascent_df5["" != ascent_df5["name"]]
ascent_df5 = ascent_df5["dont know name" != ascent_df5["name"]]
ascent_df5 = ascent_df5["no name" != ascent_df5["name"]]
ascent_df5 = ascent_df5["route" != ascent_df5["name"]]
ascent_df5 = ascent_df5["unknown" != ascent_df5["name"]]
ascent_df5 = ascent_df5["senza nome" != ascent_df5["name"]]
ascent_df5 = ascent_df5["n" != ascent_df5["name"]]
ascent_df5 = ascent_df5["via" != ascent_df5["name"]]
ascent_df5 = ascent_df5["a" != ascent_df5["name"]]
ascent_df5.name.value_counts().head(6)

dna                1584
carpe diem         1238
nirvana            1074
toma castanazo     1023
gladiator          1001
billy el rapido     972
Name: name, dtype: int64

In [89]:
ascent_df5.shape

(2719280, 23)

#### - Crag column

In [90]:
ascent_df5.crag.value_counts().head(6)

frankenjura        111402
kalymnos            95413
rodellar            57326
red river gorge     50332
margalef            46931
siurana             44608
Name: crag, dtype: int64

#### - Sector column

In [91]:
ascent_df5.sector.value_counts().head(6)

odyssey            15719
sloneczne skaly    11089
camino             10549
massone             9815
demi lune           8793
misja pec b         8329
Name: sector, dtype: int64

In [92]:
ascent_df5.sector_id.value_counts().head(6)

0.0      1036639
62.0       15718
104.0      11060
64.0        9810
63.0        8811
65.0        8736
Name: sector_id, dtype: int64

## 5. Climbers table 

We will use this table to add heigh and sex on each route of the ascent table

In [93]:
# Get the table already cleaned in the other project
climber_df = pd.read_csv('../data/climber_df.csv',low_memory=False, index_col=0)

In [94]:
climber_df = climber_df[['height','sex']].reset_index()

In [95]:
ascent_df5_2 = pd.merge(ascent_df5,climber_df,how="inner",left_on='user_id',right_on='user_id')
ascent_df5_2.head(3)

,id,user_id,grade_id,name,crag_id,crag,sector_id,sector,country,comment,rating,yellow_id,climb_try,repeat,user_recommended,chipped,sentiment,comment_bool,rating_bool,first_ascent,soft,hard,traditional,height,sex
0,2,1.0,36,king and i,16596.0,railay,61.0,dums kitchen,tha,Thailand\n,0.0,255.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,0,0,177,0
1,3,1.0,36,vet ej,0.0,nya berg segl,0.0,NaN,NaN,\n,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,177,0
2,4,1.0,36,mr big,209.0,sjoanda,371.0,huvudvaggen,swe,\n,0.0,255.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,177,0


I want to check if its hard or soft for tall people, so I will create a feature called tall_recomend:
- Tall gets deffined as higher than average + 5cm / small is avg - 5cm
- If you are tall and say its soft
- If you are small and say its hard

In [96]:
mean_height = ascent_df5_2.height.mean()

In [97]:
ascent_df5_2['tall_recommend'] = ascent_df5_2.apply(lambda x: clean_jor.Tall_recommend(x,mean_height), axis = 1)

In [98]:
ascent_df5_2.shape

(1402378, 26)

In [99]:
ascent_df5_2.to_csv('../data/ascent_df5_2.csv')
# ascent_df5_2 = pd.read_csv('../data/ascent_df5_2.csv',low_memory=False, index_col=0)

# 6. Correcting and grouping crag names, sector names and route names

I want to correct the names, for that purpose I will use one the instance "txt" defined above from <b>class text_processor

## Crag names

In [100]:
countries = ascent_df5_2.country.value_counts(dropna = False).index
countries

Index(['esp', 'fra', 'usa', 'ita', 'pol', 'deu', 'grc', 'swe', 'aut', 'bra',
       ...
       'sur', 'prk', 'myt', 'hti', 'kir', 'kaz', 'hnd', 'flk', 'bdi', 'idn'],
      dtype='object', length=142)

In [101]:
crags_from_country = ascent_df5_2[ascent_df5_2.country == countries[60]].crag
crags_from_country.value_counts().head(6)

long dong        78
guanzailing      37
guan zai ling    11
guangzulin        4
gaunzailing       4
guanzaiing        1
Name: crag, dtype: int64

In [102]:
lst = txt.replace_similar_array(list(crags_from_country))
crags_from_country_new = pd.Series(lst)
crags_from_country_new.value_counts().head(6)

long dong       78
guanzailing     54
guangzulin       4
guan zai lin     1
dtype: int64

In [103]:
crags_from_country.index

Int64Index([ 156159,  156160,  156161,  156162,  156163,  156164,  475493,
             475536,  475537,  475538,
            ...
            1252972, 1252973, 1252974, 1252975, 1285372, 1391598, 1391607,
            1391608, 1391609, 1391610],
           dtype='int64', length=137)

In [104]:
ascent_copy = ascent_df5_2.copy()
ascent_copy.loc[crags_from_country.index,'crag'] = lst

In [105]:
ascent_copy.loc[156166,:].head(6)

id           175544
user_id      4990.0
grade_id         55
name          birdy
crag_id     11594.0
crag         chorro
Name: 156166, dtype: object

We put all together in a for loop... 

This option is very time consuming, check `GUIDE.text_processor_class`and `extra.text_process_location` for more details

In [106]:
# Pre filter
ascent_df5_2.name = ascent_df5_2.name.astype(str)
ascent_df5_2.crag = ascent_df5_2.crag.astype(str)
ascent_df5_2.sector = ascent_df5_2.sector.astype(str)

ascent_df5_2 = ascent_df5_2[ascent_df5_2.name.apply(lambda x: len(x) > 3)]
ascent_df5_2 = ascent_df5_2[ascent_df5_2.crag.apply(lambda x: len(x) > 3)]
ascent_df5_2 = ascent_df5_2[ascent_df5_2.sector.apply(lambda x: len(x) > 3)]

## Clean crag

In [107]:
# %%time
# ascent_df5_3 = ascent_df5_2.copy()
# for country in countries:
# #     print(country)
#     crags_from_country = ascent_df5_3[ascent_df5_3.country == country].crag
#     lst = txt.replace_frequent_with_table(list(crags_from_country))
#     ascent_df5_3.loc[crags_from_country.index,'crag'] = lst 
# ascent_df5_3.to_csv('../data/ascent_df5_3.csv')
'''
The piece of code above takes:
CPU times: user 7min 44s, sys: 704 ms, total: 7min 45s
Wall time: 7min 47s
'''

ascent_df5_3 = pd.read_csv('../data/ascent_df5_3.csv',low_memory=False, index_col=0)

CPU times: user 7min 34s, sys: 973 ms, total: 7min 35s
Wall time: 7min 38s


In [108]:
display(ascent_df5_3.head(3))
ascent_df5_3.shape

,id,user_id,grade_id,name,crag_id,crag,sector_id,sector,country,comment,rating,yellow_id,climb_try,repeat,user_recommended,chipped,sentiment,comment_bool,rating_bool,first_ascent,soft,hard,traditional,height,sex,tall_recommend
0,2,1.0,36,king and i,16596.0,railay,61.0,dums kitchen,tha,Thailand\n,0.0,255.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,0,0,177,0,0
2,4,1.0,36,mr big,209.0,sjoanda,371.0,huvudvaggen,swe,\n,0.0,255.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,177,0,0
3,5,1.0,36,tak ska du ha,209.0,sjoanda,371.0,huvudvaggen,swe,\n,0.0,255.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,177,0,0


(1010803, 26)

### Group crag

In [109]:
# multiple columns
ascent_df6 = ascent_df5_2.copy()
ascent_df6 = ascent_df6.groupby(['country','crag','sector','name']).agg( 
    ascents_count = ('grade_id', 'count'),
    grade_sum = ('grade_id', 'sum'),
    sentiment_sum = ('sentiment','sum'),
    sentiment_count = ('comment_bool','sum'),
    rating_sum = ('rating', 'sum'),
    rating_count = ('rating_bool', 'sum'),
    repeat_sum = ('repeat','sum'),
    recommend_sum = ('user_recommended','sum'),
    chiped_sum = ('chipped','sum'),
    soft_sum = ('soft','sum'),
    hard_sum = ('hard','sum'),
    traditional_sum = ('traditional','sum'),
    tall_recommend_sum = ('tall_recommend','sum'),
    sex_sum = ('sex','sum'),
    yellow_id_sum = ('yellow_id','sum'),
    first_ascent_sum = ('first_ascent','sum'))

ascent_df6.reset_index(inplace = True)
print(ascent_df6.shape)
ascent_df6.head(5)

(269698, 20)


,country,crag,sector,name,ascents_count,grade_sum,sentiment_sum,sentiment_count,rating_sum,rating_count,repeat_sum,recommend_sum,chiped_sum,soft_sum,hard_sum,traditional_sum,tall_recommend_sum,sex_sum,yellow_id_sum,first_ascent_sum
0,abw,adlitzgraben,lari fari,fenrir,10,489,0.0,3,17.0,7,0.0,0.0,0.0,1,0,0,0,0,0.0,0
1,alb,atxarte,amilla,plaka desplome,1,49,0.0,0,3.0,1,0.0,0.0,0.0,0,0,0,0,0,0.0,0
2,alb,atxarte,puerta,andrakila,1,44,0.0,0,0.0,0,0.0,0.0,0.0,0,0,0,0,0,0.0,0
3,alb,atxarte,puerta,ley del bombero,1,49,0.0,0,1.0,1,0.0,0.0,0.0,0,0,0,0,0,0.0,0
4,alb,atxarte,puerta,marcando paquetillo celta,1,51,0.0,0,3.0,1,0.0,0.0,0.0,0,0,0,0,0,0.0,0


In [110]:
# ascent_df6.to_csv('../data/ascent_df6.csv')
# ascent_df6 = pd.read_csv('../data/ascent_df6.csv',low_memory=False, index_col=0)

In [111]:
ascent_df6[ascent_df6.country == 'esp'].crag.value_counts()

montserrat             2737
margalef               2235
pedriza                2110
siurana                1938
rodellar               1661
                       ... 
castrojo                  1
campana                   1
lobadiz                   1
san llorent de munt       1
pedrera                   1
Name: crag, Length: 2029, dtype: int64

## Clean sector

In [112]:
## Prefilter
counts = ascent_df6.sector.value_counts()[ascent_df6.sector.value_counts() > 20] # At least 20 routes per sector
ascent_df6 = ascent_df6[ascent_df6.sector.apply(lambda x: x in list(counts.index))]

In [113]:
display(ascent_df6.head(3))
ascent_df6.shape

,country,crag,sector,name,ascents_count,grade_sum,sentiment_sum,sentiment_count,rating_sum,rating_count,repeat_sum,recommend_sum,chiped_sum,soft_sum,hard_sum,traditional_sum,tall_recommend_sum,sex_sum,yellow_id_sum,first_ascent_sum
2,alb,atxarte,puerta,andrakila,1,44,0.0,0,0.0,0,0.0,0.0,0.0,0,0,0,0,0,0.0,0
3,alb,atxarte,puerta,ley del bombero,1,49,0.0,0,1.0,1,0.0,0.0,0.0,0,0,0,0,0,0.0,0
4,alb,atxarte,puerta,marcando paquetillo celta,1,51,0.0,0,3.0,1,0.0,0.0,0.0,0,0,0,0,0,0.0,0


(121049, 20)

In [114]:
# %%time
# ascent_df7 = ascent_df6.copy()
# for country in countries:
# #     print(country)
#     crags = ascent_df7[ascent_df7.country == country].crag.value_counts(dropna = False).index
    
#     for crag in crags:  
#         sectors_from_crag = ascent_df7[ascent_df7.crag == crag].sector       
#         lst = txt.replace_frequent_with_table(list(sectors_from_crag), show = False)
#         ascent_df7.loc[sectors_from_crag.index,'sector'] = lst 
        
# ascent_df7.to_csv('../data/ascent_df7.csv')
'''
The piece of code above takes:
CPU times: user 1min 17s, sys: 116 ms, total: 1min 17s
Wall time: 1min 17s
'''

ascent_df7 = pd.read_csv('../data/ascent_df7.csv',low_memory=False, index_col=0)

CPU times: user 1min 8s, sys: 112 ms, total: 1min 8s
Wall time: 1min 8s


In [115]:
print(ascent_df7.shape)
ascent_df7.head(3)

(121049, 20)


,country,crag,sector,name,ascents_count,grade_sum,sentiment_sum,sentiment_count,rating_sum,rating_count,repeat_sum,recommend_sum,chiped_sum,soft_sum,hard_sum,traditional_sum,tall_recommend_sum,sex_sum,yellow_id_sum,first_ascent_sum
2,alb,atxarte,puerta,andrakila,1,44,0.0,0,0.0,0,0.0,0.0,0.0,0,0,0,0,0,0.0,0
3,alb,atxarte,puerta,ley del bombero,1,49,0.0,0,1.0,1,0.0,0.0,0.0,0,0,0,0,0,0.0,0
4,alb,atxarte,puerta,marcando paquetillo celta,1,51,0.0,0,3.0,1,0.0,0.0,0.0,0,0,0,0,0,0.0,0


### Group sector

In [116]:
# We group again
# multiple columns
ascent_df9 = ascent_df7.copy()
ascent_df9 = ascent_df9.groupby(['country','crag','sector','name']).agg( 
    ascents_count = ('ascents_count', 'sum'),
    grade_sum = ('grade_sum', 'sum'), 
    sentiment_sum = ('sentiment_sum','sum'),
    sentiment_count = ('sentiment_count','sum'),
    
    rating_sum = ('rating_sum', 'sum'),
    rating_count = ('rating_count','sum'),
    
    repeat_sum = ('repeat_sum','sum'),
    recommend_sum = ('recommend_sum','sum'),
    chiped_sum = ('chiped_sum','sum'),
    soft_sum = ('soft_sum','sum'),
    hard_sum = ('hard_sum','sum'),
    traditional_sum = ('traditional_sum','sum'),
    tall_recommend_sum = ('tall_recommend_sum','sum'),
    sex_sum = ('sex_sum','sum'),
    
    yellow_id_sum = ('yellow_id_sum','sum'),
    first_ascent_sum = ('first_ascent_sum','sum'))

# # Filter more than 3 ascents per route
ascent_df9 = ascent_df9[ascent_df9.ascents_count > 1]
ascent_df9.reset_index(inplace = True)

ascent_df9.reset_index(inplace = True)

In [117]:
print(ascent_df9.shape)

(59114, 21)


Now we do the cleaning for the <b>Route names..

## Clean route names

In [118]:
# %%time
# ascent_df10 = ascent_df9.copy()
# for country in countries:
# #     print(country)
#     crags = ascent_df10[ascent_df10.country == country].crag.value_counts(dropna = False).index
    
#     for crag in crags:  
#         sectors = ascent_df10[ascent_df10.crag == crag].sector.value_counts(dropna = False).index
        
#         for sector in sectors:           
#             names_from_sector = ascent_df10[ascent_df10.sector == sector].name
#             lst = txt.replace_frequent_with_table(list(names_from_sector), show = False)
#             ascent_df10.loc[names_from_sector.index,'name'] = lst 
        
# ascent_df10.to_csv('../data/ascent_df10.csv')
'''
The piece of code above takes:
CPU times: user 5min 4s, sys: 152 ms, total: 5min 4s
Wall time: 5min 5s
'''

ascent_df10 = pd.read_csv('../data/ascent_df10.csv',low_memory=False, index_col=0)

CPU times: user 4min 18s, sys: 77 ms, total: 4min 18s
Wall time: 4min 18s


### Group route names

In [119]:
# We group again
# multiple columns
ascent_df11 = ascent_df10.copy()
ascent_df11 = ascent_df11.groupby(['country','crag','sector','name']).agg( 
    ascents_count = ('ascents_count', 'sum'),
    grade_sum = ('grade_sum', 'sum'), 
    sentiment_sum = ('sentiment_sum','sum'),
    sentiment_count = ('sentiment_count','sum'),
    
    rating_sum = ('rating_sum', 'sum'),
    rating_count = ('rating_count','sum'),
    
    repeat_sum = ('repeat_sum','sum'),
    recommend_sum = ('recommend_sum','sum'),
    chiped_sum = ('chiped_sum','sum'),
    soft_sum = ('soft_sum','sum'),
    hard_sum = ('hard_sum','sum'),
    traditional_sum = ('traditional_sum','sum'),
    tall_recommend_sum = ('tall_recommend_sum','sum'),
    sex_sum = ('sex_sum','sum'),
    
    yellow_id_sum = ('yellow_id_sum','sum'),
    first_ascent_sum = ('first_ascent_sum','sum'))

# # Filter more than 3 ascents per route
ascent_df11 = ascent_df11[ascent_df11.ascents_count > 1]
ascent_df11.reset_index(inplace = True)

In [120]:
print(ascent_df11.shape)

(55858, 20)


In [121]:
ascent_df11

,country,crag,sector,name,ascents_count,grade_sum,sentiment_sum,sentiment_count,rating_sum,rating_count,repeat_sum,recommend_sum,chiped_sum,soft_sum,hard_sum,traditional_sum,tall_recommend_sum,sex_sum,yellow_id_sum,first_ascent_sum
0,and,montserrat,prohibitivo,diagonal de la x,4,197,0.0000,2,6.0,3,0.0,0.0,0.0,0,1,0,-1,0,0.0,0
1,and,montserrat,prohibitivo,mehir,6,294,0.0000,3,10.0,5,0.0,1.0,0.0,1,1,0,-1,0,0.0,0
2,and,montserrat,prohibitivo,pas de la discordia,5,245,0.0000,5,7.0,4,0.0,1.0,0.0,0,0,0,0,0,0.0,0
3,and,tartareu,bombo suis,tenedor libre,3,133,0.0000,2,3.0,1,0.0,1.0,0.0,0,1,0,0,0,0.0,0
4,arg,bandurrias,rincon,tendinitis,2,97,0.0762,1,1.0,1,0.0,0.0,0.0,1,0,0,1,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55853,zaf,van stadens,cleopatra wall,steel sunday,3,120,0.4753,1,0.0,0,0.0,2.0,1.0,0,0,0,0,0,765.0,0
55854,zaf,van stadens,cleopatra wall,suck it up princess,2,88,0.7074,1,0.0,0,0.0,0.0,0.0,0,0,0,0,0,255.0,0
55855,zaf,van stadens,cleopatra wall,sunset sonata,2,109,0.8636,1,3.0,1,0.0,0.0,0.0,1,0,0,1,0,510.0,1
55856,zaf,van stadens,cleopatra wall,tekel,2,88,0.0000,0,2.0,1,0.0,1.0,0.0,0,0,0,0,0,0.0,0


#### Create new features and remove others...

In [122]:
# Sex sum is not interesting, but sex_sum/ascents_count it gets the ratio men/wmen climbing that route 0men/1women

ascent_df11['grade_mean'] = ascent_df11.grade_sum/ascent_df11.ascents_count
ascent_df11['sentiment_mean'] = ascent_df11.sentiment_sum/ascent_df11.sentiment_count
ascent_df11['rating_mean'] = ascent_df11.rating_sum/ascent_df11.rating_count
ascent_df11['sex_ratio'] = ascent_df11.sex_sum/ascent_df11.ascents_count
ascent_df11['yellow_id_mean'] = ascent_df11.yellow_id_sum/ascent_df11.ascents_count
ascent_df11['first_ascent_mean'] = ascent_df11.first_ascent_sum/ascent_df11.ascents_count

ascent_df11 = ascent_df11.drop(columns = ['grade_sum','sentiment_sum','sentiment_count',
                                          'rating_sum','rating_count','sex_sum','yellow_id_sum','first_ascent_sum'])

print(ascent_df11.shape)
ascent_df11.tail(5)

(55858, 18)


,country,crag,sector,name,ascents_count,repeat_sum,recommend_sum,chiped_sum,soft_sum,hard_sum,traditional_sum,tall_recommend_sum,grade_mean,sentiment_mean,rating_mean,sex_ratio,yellow_id_mean,first_ascent_mean
55853,zaf,van stadens,cleopatra wall,steel sunday,3,0.0,2.0,1.0,0,0,0,0,40.0,0.4753,NaN,0.0,255.0,0.0
55854,zaf,van stadens,cleopatra wall,suck it up princess,2,0.0,0.0,0.0,0,0,0,0,44.0,0.7074,NaN,0.0,127.5,0.0
55855,zaf,van stadens,cleopatra wall,sunset sonata,2,0.0,0.0,0.0,1,0,0,1,54.5,0.8636,3.0,0.0,255.0,0.5
55856,zaf,van stadens,cleopatra wall,tekel,2,0.0,1.0,0.0,0,0,0,0,44.0,NaN,2.0,0.0,0.0,0.0
55857,zaf,waterval boven,baboon buttress,pretenders,2,0.0,1.0,0.0,0,0,0,0,49.0,0.8020,NaN,0.0,255.0,0.0


The routes will be unique, but we only have the route_name as Id, I will asign a route_id to each route based on the name and the crag_id:

In [123]:
ascent_df12 = ascent_df11.copy()
ascent_df12.index.names = ['name_id']
ascent_df12.reset_index(inplace=True)

In [124]:
# And I will create the features table
routes_features = ascent_df12

In [125]:
routes_features.to_csv('../data/routes_features.csv')

In [131]:
climb_jor.Route_search(ascent_df11,'ultravox')

,country,crag,sector,name,ascents_count,repeat_sum,recommend_sum,chiped_sum,soft_sum,hard_sum,traditional_sum,tall_recommend_sum,grade_mean,sentiment_mean,rating_mean,sex_ratio,yellow_id_mean,first_ascent_mean
19122,esp,montserrat,vermell,ultravox,23,0.0,4.0,0.0,5,2,0,3,56.173913,-0.163376,2.85,0.130435,232.826087,0.0
